In [3]:
__author__ = "Cindy Wang"
__version__ = "Alibaba August 2018"

# Setup

The following packages are required:
```
numpy==1.14.0
scipy==1.0.0
scikit-learn >= 0.19.1
jupyter==1.0.0
```

You can paste these into a `requirements.txt` file and run
`pip install -r requirements.txt`. You will also need to install Tensorflow by following the instructions on the [website](https://www.tensorflow.org/install/).

## Running the notebook
Navigate to your notebooks directory (wherever this file is located) and run `jupyter notebook --port 5656` to start the notebook server.

# RNNs

Recall the most basic artifical neural network, an MLP (multi-layer perceptron, or feed-forward neural network). Each layer of this network has its own weights $W$, biases $b$, and activation function $f$. These are applied, then the activations are sent to the next layer. The following set of equations describes an $N$-layer MLP:

$$\begin{align} 
A_0 : h_0 &= f(W_0 x + b_0) \\ 
A_1 : h_1 &= f(W_1 h_0 + b_1) \\ 
&\dots \\
A_N : h_N &= f(W_N h_{N-1} + b_N)
\end{align}$$

Now imagine that instead of a single vector input, we have a sequence of vectors. Our objective is now to capture relationships between successive inputs. We can achieve this by feeding **successive inputs** to **successive hidden layers**:
![rnn1](img/rnn1.png)

In a typical MLP, the weights $W$ and biases $b$ in each layer are independent. However, if we want the neural network layers $A$ to remember the state of previous inputs when reading the next input in the sequence, we can roll the $A_n$s into a single layer $A$. At each time step, the inputs are supplied to the same network layer $A$ with a single set of weights and biases -- this $A$ is referred to as the **RNN cell**. The outputs of the RNN are just the hidden states $h_t$ at each time step. This gives us the classic RNN:
![rnn2](img/rnn2.png)

You might have noticed that unlike the MLP hidden layer which has a single input, an RNN cell has two inputs: $x_t$ and the previous hidden state $h_{t-1}$. Therefore, we replace the weight matrix $W$ with two different matrices $U$ and $V$, which are applied to $x_t$ and $h_{t-1}$ respectively. Notice the differences between the MLP and RNN layers:

$$\begin{align*} 
& \textbf{MLP: } & & \textbf{RNN: }\\
A_0 : h_0 &= f(W_0 x + b_0) &  h_0 &= f(Vx_0 + b)\\ 
A_1 : h_1 &= f(W_1 h_0 + b_1) & h_1 &= f(Uh_0 + Vx_1 + b)\\ 
& \dots & &\dots\\
A_N : h_N &= f(W_N h_{N-1} + b_N) & h_N &= f(Uh_{N-1} + Vx_N + b)
\end{align*}$$

## The problem with RNNs

We now have a good understanding of the basic vanilla RNN. However, RNNs of the form described above aren't used very often in the real world. Why? The issue is something called the **vanishing gradient problem**. 

In practice, we want our RNN to be able to handle a large number of time steps and remember inputs across long distances. For example, consider the following sentences:

1. "Jane walked into the room. John walked in too. Jane said hi to ___"
2. "Jane walked into the room. John walked in too. It was late in the day, and everyone was walking home after a long day at work. Jane said hi to ___"

Clearly, the next word in both sentences should be "John." However, a vanilla RNN is less likely to correctly predict the next word in Sentence 2. This is because of the way RNN weights are updated during training via a method called **backpropagation**: the training updates to the weights gradually vanish as they propagate to earlier time steps.

Let us briefly discuss the mathematical reasoning behind this. Consider the equations for an RNN with $N$ time steps (note that we will leave out the biases from here on for clarity):

$$\begin{align}
h_0 &= f(Vx_0)\\ 
h_1 &= f(Uh_0 + Vx_1)\\ 
& \dots \\
h_N &= f(Uh_{N-1} + Vx_N)\\
out &= output\_layer(h_N)\\
J &= loss\_fn(out, label)
\end{align}$$

Essentially, backpropagation involves calculating the gradient (partial derivative) of the total error with respect to each of the weights in the network so that we can update them using gradient descent. Say we want to find the gradient with respect to the weight matrix $U$. You might remember from calculus that we can combine partial derivatives using the chain rule. The gradient of the error $J$ with respect to $U$ then looks something like:

$$\frac{\partial J}{\partial U} = \frac{\partial J}{\partial out}\frac{\partial out}{\partial h_N}\frac{\partial h_N}{\partial h_{N-1}} \dots \frac{\partial h_1}{\partial h_0}\frac{\partial h_0}{\partial U}$$

The vanishing gradient problem emerges as $N$ increases, since each partial gradient term involves calculating the gradient of the activation function $f$. Consider the plot below of the gradient for $f=\sigma$, the sigmoid function:

![sigmoid gradient](img/Sigmoid-gradient.png)

You can observe that the values of the gradient $f'(x)$ (shown in orange) are always <0.25 and become very small when $f(x)$ is close to 0 or 1. In order to get $\frac{\partial J}{\partial V}$, we need to multiply several such gradients together. The resulting gradient becomes close to zero, or "vanishes." As a result, our network is not able to update its weights across a large number of time steps.

This is only a rough sketch of what goes on during backpropagation, but it is sufficient to understand why we need something more complex than a vanilla RNN. For a full explanation of the vanishing and exploding gradient problem, the original paper is by [Bengio et al., 1994](https://ieeexplore.ieee.org/document/279181/).

# LSTMs

The LSTM, or long short-term memory network, is a specialized RNN cell that has a few advantages over the vanilla RNN cell:
- It keep tracks of an *internal memory state* in addition to a new *candidate state* at each time step
- It uses gates to control how much of the internal state to forget and how much of the candidate state to add
- It can remember long-term memory
- It reduces the vanishing gradient problem

As a result, LSTMs are used widely in practical settings and are a strong baseline for many sequence-related machine learning tasks. We will now take a closer look at how they work.

Notice that the LSTM is a generalized form of the vanilla RNN. They have the same chainlike structure, but the LSTM cell contains more layers, all of which interact together:.

**RNN:** ![rnn3](img\rnn3.png) 
**LSTM:** ![lstm1](img\lstm1.png) 

Without worrying too much about the variable names for now, compare the equations for a cell at timestep $t$:

\begin{aligned}
    & \textbf{RNN: }(f = tanh) & & \textbf{LSTM: }\\
    h_{t} &= \tanh\big(Uh_{t-1} + Vx_{t}\big) & \tilde{C}_{t} &= \tanh\big(U^{g}h_{t-1}+V^{g}x_{t}\big) \\
    && f_{t} & =\sigma\big(U^{f}h_{t-1}+V^{f}x_{t}\big) \\
    && i_{t} & =\sigma\big(U^{i}h_{t-1}+V^{i}x_{t}\big) \\
    && o_{t} & =\sigma\big(U^{o}h_{t-1}+V^{o}x_{t}\big) \\
    && C_{t} & =\sigma\big(f_{t}\ast C_{t-1}+i_{t}\ast\tilde{C}_{t}\big) \\
    && h_{t} & =\tanh(C_{t})\ast o_{t}
\end{aligned}

We see that the LSTM candidate state $\tilde{C}_{t}$, is identical to the RNN hidden state $h_t$! This is not a coincidence. While a vanilla RNN directly returns this value as its hidden state, the LSTM combines $\tilde{C}_{t}$ with 4 other values -- $i_t, f_t, o_t, C_t$ -- in order to get its hidden state.

First, we explain what each of these variables mean:

### States
- cell state $C_t$: The internal memory of the LSTM. It gets updated with linear modifications at each step.
- candidate state $\tilde{C}_{t}$: The "new" state at each time step. A portion of this is saved to the cell state.
- hidden state $h_{t}$: The output of the LSTM.

### Gates
A gate consists of a feed-forward layer with a sigmoid activation. It takes $h_{t-1}$ and $x_t$ as inputs and outputs a number between 0 and 1 for each state dimension. These numbers denote how much information to let through the gate.
- forget gate $f_t$: What part of the previous cell state $C_{t-1}$ to retain.
- input gate $i_t$: What part of the candidate state $\tilde{C}_{t}$ to add.
- output gate $o_t$: What part of the cell state $C_{t}$ to output.

## Step-by-step
Next, we walk through the LSTM equations step by step. The inputs to the cell are the previous cell state $C_{t-1}$, the previous hidden state $h_{t-1}$, and the input vector $x_t$ (i.e. the $t$th word in the sequence). 

#### Retain old info
Our first step is to figure out what information to retain from the previous cell state. The forget gate $f_{t}$ looks at $h_{t-1}$ and $x_t$ and outputs a number between 0 and 1 for each dimension of $C_{t-1}$. A 1 means to completely retain the old state and a 0 means to completely throw it away. We then multiply $f_{t}$ pointwise with $C_{t-1}$ to get the previous state's contribution.

\begin{aligned}
f_{t} &=\sigma\big(U^{f}h_{t-1}+V^{f}x_{t}\big)\\
old\_info_t &= f_{t}\ast C_{t-1}
\end{aligned}

#### Add new info
Our next step to do similar calculations to find the contribution of new information. We calculate the candidate state $\tilde{C}_{t}$ (which, as we noticed before, is same as finding the vanilla RNN hidden state). We then calculate the input gate $i_t$ and pointwise multiply the two to get the candidate state's contribution.

\begin{aligned}
\tilde{C}_{t} &= \tanh\big(U^{g}h_{t-1}+V^{g}x_{t}\big)\\
i_{t} &= \sigma\big(U^{i}h_{t-1}+V^{i}x_{t}\big)\\
new\_info_t &= i_{t}\ast\tilde{C}_{t}
\end{aligned}

#### Combine
Now that we have the previous cell state and current candidate state's contributions, we can combine them to update the cell state.

![lstm3-focus-C](img\LSTM3-focus-C.png)

\begin{aligned}
C_{t} &= \sigma\big(old\_info_t + new\_info_t) \\
&= \sigma\big(f_{t}\ast C_{t-1}+i_{t}\ast\tilde{C}_{t}\big) \\
\end{aligned}

#### Output
Finally, we need to decide what part of this cell state we actually want to output as $h_t$. The reason that we do this instead of directly outputting $C_t$ is because it allows us to selectively output relevant information. Take for example translation to English: if the current word is a noun, we may want to output whether it is singular or plural so we know what form a verb should take if it comes next.

For the output, we apply tanh to the cell state (to constrain the values between -1 and 1), calculate the output gate $o_t$, and pointwise multiply to get $h_t$.

![lstm3-focus-o](img\LSTM3-focus-o.png)

$$o_{t} =\sigma\big(U^{o}h_{t-1}+V^{o}x_{t}\big)$$
$$h_{t} =\tanh(C_{t})\ast o_{t}$$

Here is a diagram of the entire LSTM cell with labeled components for reference.
![lstm2](img\lstm2.png)

## Implementing LSTMs in Tensorflow

We just took the time to explain the LSTM equations, but in practice you will rarely have to declare the weight matrices and operations yourself. Instead, Tensorflow provides an interface to declare a generic RNN with your choice of cell:

In [ ]:
import tensorflow as tf

lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units) # Can also be RNN, GRU, or custom
outputs, states = tf.nn.dynamic_rnn(lstm_cell, inputs) # [h_0 ... h_N], (C_N, h_N)

There are some additional details to actually getting an LSTM up and running in Tensorflow, so we will now go though a full working example with Tensorflow code.

# LSTM Encoder-Decoder: A simple working example

## Background

### Sequence-to-Sequence
For many NLP tasks, the output is a classification:
- Sentiment Analysis
    - **in:** "That was a good movie"
    - **out:** [negative, neutral, **positive**]
- Natural Language Inference
    - **in:** ("A black race car starts up in front of a crowd of people.", "A man is driving down a lonely road.")
    - **out:** [entailment, neutral, **contradiction**]

However, there are some tasks that where the output is a sequence and must be generated instead of selected. *Sequence-to-sequence* describes a class of models built for tasks where the input and output are both sequences. These typically require more complex modeling to capture an additional dimension of information.
- Machine Translation
    - **in:** "Do you like to play badminton?"
    - **out:** "你喜欢打羽毛球吗?"
- Sequence Labeling
    - **in:** 
    `"John  lives in New   York  and works for the European Union"`
    - **out:** 
    `B-PER O     O  B-LOC I-LOC O   O     O   O   B-ORG    I-ORG`
- Video Captioning

### Encoder-Decoder
*Encoder-decoder* describes a type of model architecture where the entire input is first *encoded* into a single vector, then decoded into the desired output. Many sequence-to-sequence models have this structure, which is useful for tasks with variable length outputs and/or where the entire input is needed in order to generate the output.
- Question Answering
    - **in:** 
        - P: " Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals **within a cloud**."
        - Q: "Where do water droplets collide with ice crystals to form precipitation?"
    - **out:** 
        - A: **within a cloud**
- Text Summarization
    - **in:** "Alice and Bob took the train to visit the zoo. They saw a baby giraffe, a lion, and a flock of colorful tropical birds.
    - **out:** "Alice and Bob visited the zoo and saw animals and birds."

## Task
We will go through a basic example that involves feeding a sequence of IDs (numbers) into an LSTM one at a time. (For NLP, these IDs would be the IDs of words in the vocabulary.) Our goal is for the network to echo back a partial or full list of contiguous observations observed.

While this seems overly simplistic, we can think of it as a basic form of machine translation. This task requires the network to remember blocks of contiguous observations and demonstrates the LSTM's ability to encode temporal information in low dimensions. We will achieve this using an encoder-decoder sequence-to-sequence model, which is the state of the art architecture for NMT.

**Input => Output**
```
[86, 81, 88, 1, 23] => [86, 81]
[78, 64, 7, 99, 23] => [78, 64]
[2, 36, 73, 26, 27] => [2, 36]
...
[13, 13, 53, 40, 64] => [13, 13]
```

## Architecture
![enc-dec-2](img\enc-dec-2.jpg)

In [3]:
# This cell is runnable!

import tensorflow as tf
import numpy as np
import pandas as pd
import random
import sys
from sklearn.model_selection import train_test_split

/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._trave

/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/scipy/stats/_continuous_distns.py:18: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _stats
/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/sklearn/utils/extmath.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms
/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutu

## Generate datasets

Since it is not important what the IDs themselves are, we will just generate them randomly. We will arbitrarily set the following parameters:

- *input_dim* = 100: number of possible IDs, 100 means range is 0-99
- *n_in* = 5: number of words in each input sequence
- *n_out* = 2: number of words to echo back (output)

We set aside 20% of the data for testing and use the rest for training.

In [7]:
# This cell is runnable!

def generate_sequence(input_dim, num_seq, length):
    return np.random.randint(input_dim, size=(num_seq, length))

def get_data(input_dim, n_in, n_out, size=10000):
    X = generate_sequence(input_dim, size, n_in)
    y = X[:,:n_out]
    return X, y

input_dim = 100
n_in = 5
n_out = 2
X, y = get_data(input_dim, n_in, n_out)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print "Sample X:\n", X_train[:5]
print "Sample y:\n", y_train[:5]

Sample X:
[[40 34 57 60 27]
 [65 93 17 47 81]
 [13 29 74 47 89]
 [77 17 92 19  2]
 [48 75 74 19 14]]
Sample y:
[[40 34]
 [65 93]
 [13 29]
 [77 17]
 [48 75]]


## Embed sequence
* Before shape: [*n_in*]
* After shape: [*n_in* x *input_dim*]

In the previous lesson, we learned about word embeddings. A word embedding is a fixed dimensional vector that represents one element of the input sequence. If our IDs represented words in a large vocabulary, we would want to use pre-trained word embeddings such as word2vec/GloVe, or initialize them to random vectors and let our model train them.

However, since our vocabulary is small (IDs from 0-99), here we will use one-hot vectors. This means we represent ID $n$ with a vector that is 1 at position $n$ and 0 everywhere else.

```
[0,2,1] -> [[1., 0., 0.],
            [0., 0., 1.],
            [0., 1., 0.]]
```

In [135]:
def to_embedding_random(ids, vocab_size, embedding_size):
    with tf.variable_scope('word_embedding'):
        # If GloVe or word2vec, C would be need to be passed in instead of declared as a variable
        C = tf.get_variable('C', [vocab_size, embedding_size], initializer=tf.random_normal_initializer())
        return tf.nn.embedding_lookup(C, ids)
                        
def to_embedding_one_hot(ids, vocab_size):
    return tf.one_hot(ids, vocab_size)

inputs = to_embedding_one_hot(X_train, input_dim)
outputs = to_embedding_one_hot(y_train, input_dim)

## Build model

### Encoder
* Before shape: [*n_in* x *input_dim*]
* After shape: [1 x *hidden_dim*]

After the embedding step, our input is *n_in* vectors of length *input_dim*. We will use an LSTM encoder to *encode* all the information from the input sequence into a single vector of length *hidden_dim*.

Remember that the LSTM keeps track of its internal memory state $C_t$. We will feed our entire input $w_1, w_2, \dots w_{n_{in}}$ into an LSTM and extract the LSTM state at the final step, $C_{n_{in}}$, as our encoding.

In [ ]:
hidden_dim = 150
encoder_cell = tf.nn.rnn_cell.LSTMCell(hidden_dim)
encoder_h, encoder_c_and_h = tf.nn.dynamic_rnn(
    encoder_cell,
    inputs,
    dtype=tf.float32)

### Decoder
* Before shape: [1 x *hidden_dim*]
* After shape: [*n_out* x *hidden_dim*]

Now we need to decode our encoding into a sequence of *n_out* IDs. We will run another LSTM for $n_{out}$ time steps and extract the hidden states $h_1 \dots h_{n_{out}}$ as our outputs. We will simply use the encoder final state as the input at each of the time steps. We also use the encoder final state to initialize the decoder LSTM state.

In [ ]:
# Copy encoder state n_out times
decoder_inputs = tf.tile(tf.expand_dims(encoder_c_and_h.h, 1), (1, n_out, 1))

# Construct decoder LSTM with encoder state as initial state
decoder_cell = tf.nn.rnn_cell.LSTMCell(hidden_dim)
decoder_h, decoder_c_and_h = tf.nn.dynamic_rnn(
    decoder_cell, 
    decoder_inputs, 
    initial_state=encoder_c_and_h)

### Output layer
* Before shape: [*n_out* x *hidden_dim*]
* After shape: [*n_out* x *input_dim*]

After the decoding, all we need to do is use a feed-forward layer (like the MLP) to transform our outputs from *hidden_dim* to *input_dim* dimensions. We now have a sequence of word embeddings again! We then apply a softmax over the word embedding dimension to get a probability distribution over the IDs, and predict the ID with the highest probability for each element in the sequence.

In [ ]:
logits = tf.layers.dense(decoder_h, input_dim)
predictions = tf.argmax(tf.nn.softmax(logits), axis=2)

## Train model

We will use cross entropy loss to train our model. We use the Adam optimizer, which is a variant of stochastic gradient descent that uses a dynamic learning rate.

Above, we built our model graph -- `model_outputs` represents the final output of our model. In order to train it, we now need to define a loss and optimizer. Tensorflow optimizers automatically compute gradients for a loss and apply gradients to variables.

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=outputs))
optimizer = tf.train.AdamOptimizer(eta).minimize(cost) # eta is learning rate

# Run the training operation
for i in range(iters):
    _, loss, predictions = self.sess.run([optimizer, cost, predictions])

Here is the complete code for a `BasicEncoderDecoder` class, followed by a simple example script for how to use it. Like a `scikit-learn` model, we first create a model object with hyperparameters, then directly call `fit(X,y)` and `predict(X,y)` on it.

In [10]:
# This cell is runnable!

class BasicEncoderDecoder(object):
    """
    Adapted from Chris Potts's CS224U model classes (https://github.com/cgpotts/cs224u)
    
    Parameters
    ----------
    embed_dim : int
        Dimensionality of embeddings (if one-hot, size of vocab).
    cell_class : tf.nn.rnn_cell class
       The default is `tf.nn.rnn_cell.LSTMCell`. Other prominent options:
       `tf.nn.rnn_cell.BasicRNNCell`, and `tf.nn.rnn_cell.GRUCell`.
    hidden_dim : int
        Dimensionality of hidden layers.
    hidden_activation : tf.nn activation
       E.g., tf.nn.relu, tf.nn.relu, tf.nn.selu.
    max_iter : int
    eta : float
        Learning rate
    tol : float
        Stopping criterion for the loss.
    display_progress : int
        For value i, progress is printed every ith iteration.
    """
    def __init__(self, 
                 embed_dim=100,
                 cell_class=tf.nn.rnn_cell.LSTMCell,
                 hidden_dim=150, 
                 hidden_activation=tf.nn.tanh,
                 batch_size=200, 
                 eta=0.01, 
                 tol=1e-4, 
                 display_progress=1):
        self.embed_dim = embed_dim
        self.cell_class = cell_class
        self.hidden_dim = hidden_dim
        self.hidden_activation = hidden_activation
        self.batch_size = batch_size
        self.eta = eta
        self.tol = tol
        self.display_progress = display_progress
    
    def _encoder(self, encoder_inputs):
        with tf.variable_scope("encoder"):
            # Declaring LSTM cell
            encoder_cell = self.cell_class(
                self.hidden_dim, activation=self.hidden_activation)
            # Declaring RNN with above cell
            _, encoder_state = tf.nn.dynamic_rnn(
                encoder_cell,
                encoder_inputs,
                dtype=tf.float32)
            decoder_inputs = tf.tile(tf.expand_dims(encoder_state.h, 1), (1, self.trg_len, 1))
            return encoder_state, decoder_inputs
            
    def _decoder(self, encoder_state, decoder_inputs):
        with tf.variable_scope("decoder"):
            decoder_cell = self.cell_class(
                self.hidden_dim, activation=self.hidden_activation)
            decoder_outputs, _ = tf.nn.dynamic_rnn(
                decoder_cell, 
                decoder_inputs, 
                initial_state=encoder_state)
            # Dense layer to get logits of size embed_dim
            return tf.layers.dense(decoder_outputs, self.embed_dim)
        
    def _enc_dec_body(self, inputs):
        encoder_state, decoder_inputs = self._encoder(inputs)
        return self._decoder(encoder_state, decoder_inputs)
            
    def _to_embedding(self, ids):
        # Can alternatively use random, word2vec, or GloVe embeddings
        return tf.one_hot(ids, self.embed_dim)
    
    def fit(self, X, y, max_iter=100, **kwargs):
        """Standard `fit` method.
        Parameters
        ----------
        X : [n x src_len] int
        y : [n x trg_len] int

        Returns
        -------
        self
        """
        if isinstance(X, pd.DataFrame):
            X = X.values

        self.src_len = len(X[0])
        self.trg_len = len(y[0]) 

        # Start the session:
        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()

        # Declare the inputs and outputs
        self.src = tf.placeholder(
            tf.int32, [None, self.src_len])
        self.trg = tf.placeholder(
            tf.int32, shape=[None, self.trg_len])
        inputs = self._to_embedding(self.src)
        outputs = self._to_embedding(self.trg)
        
        # Build the encoder-decoder body.
        self.model = self._enc_dec_body(inputs)

        # Optimizer set-up:
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                logits=self.model, labels=outputs))
        self.optimizer = tf.train.AdamOptimizer(self.eta).minimize(self.cost)
        self.pred = tf.argmax(tf.nn.softmax(self.model), axis=2)
        self.accuracy = tf.metrics.accuracy(self.trg, self.pred)

        # Initialize the session variables:
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer()) # Necessary for self.accuracy

        # Training, full dataset for each iteration:
        for i in range(1, max_iter+1):
            loss = 0
            acc = []
            for X_batch, y_batch in self._batch_iterator(X, y):
                _, batch_loss, batch_acc = self.sess.run(
                    [self.optimizer, self.cost, self.accuracy],
                    feed_dict={
                        self.src: X,
                        self.trg: y
                    })
                loss += batch_loss
                acc.append(batch_acc[1])
            acc = sum(acc)/len(acc)
            if loss < self.tol:
                self._progressbar("stopping with loss < self.tol", i)
                break
            else:
                self._progressbar("loss: {}, accuracy: {}".format(loss, acc), i)
        return self
    
    def _batch_iterator(self, X, y):
        dataset = list(zip(X, y))
        random.shuffle(dataset)
        for i in range(0, len(dataset), self.batch_size):
            batch = dataset[i: i+self.batch_size]
            X_batch, y_batch = zip(*batch)
            yield X_batch, y_batch
    
    def _progressbar(self, msg, index):
        if self.display_progress and index % self.display_progress == 0:
            sys.stderr.write('\r')
            sys.stderr.write('Iter {}: {}'.format(index, msg))
            sys.stderr.flush()

    def predict(self, X, y=None, report_acc=False):
        """Return target sequence.
        Parameters
        ----------
        X : [n x src_len] int
        y (optional) : [n x trg_len] int
            Must be passed if report_acc is True
        report_acc (optional) : boolean
        
        Returns
        -------
        [n x trg_len] int
         
        OR
        
        [ 
            [n x trg_len] int, 
            float 
        ]
        """
        if report_acc:
            if y is None:
                raise ValueError
            return self.sess.run(
                [self.pred, self.accuracy], feed_dict={self.src: X, self.trg: y})
        return self.sess.run([self.pred], feed_dict={self.src: X})[0]


In [11]:
# This cell is runnable!

# These are the default settings, feel free to change them and see what happens!
bed = BasicEncoderDecoder(embed_dim=100,
                 cell_class=tf.nn.rnn_cell.LSTMCell,
                 hidden_dim=150, 
                 hidden_activation=tf.nn.tanh,
                 batch_size=200, 
                 eta=0.01, 
                 tol=1e-4, 
                 display_progress=1)
bed.fit(X_train, y_train, max_iter=25)

/Users/cindywang/miniconda3/envs/tensorflow/lib/python2.7/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iter 5: loss: 0.382635515649, accuracy: 0.839400216937

KeyboardInterrupt: 

In [168]:
# This cell is runnable!

preds, acc = bed.predict(X_test, y_test, report_acc=True)
print "Accuracy: ", acc[1]

Accuracy:  0.9725975


# Conclusion

This is the end of our RNN + LSTM crash course! Today we discussed RNNs, a type of neural network that can model sequential data. We introduced the mathematical and conceptual ideas behind LSTMs, a type of RNN that specializes in long-range memory tasks. We also implemented a basic LSTM encoder-decoder model in Tensorflow and trained it to echo back a partial list of observed data.

## Further reading

There is much more to be learned on all the topics mentioned in this tutorial. Here is an incomplete list of related topics and useful articles:
- Visual explanation of LSTMs and variants (including GRU) http://colah.github.io/posts/2015-08-Understanding-LSTMs/ (**highly recommended!**)
- Neural network math (including backpropagation) http://adventuresinmachinelearning.com/neural-networks-tutorial
- Sequence to sequence learning with "teacher forcing" https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    - "Teacher forcing" means to output one token at a time (instead of all at once) and to use the network's prior outputs as inputs to the decoder. This is how most current NMT models work!
- Tensorflow RNN documentation https://www.tensorflow.org/api_guides/python/contrib.rnn

### References
- "Fundamentals of Deep Learning – Introduction to Recurrent Neural Networks" https://www.analyticsvidhya.com/blog/2017/12/introduction-to-recurrent-neural-networks/
- "Neural Networks Tutorial – A Pathway to Deep Learning" http://adventuresinmachinelearning.com/neural-networks-tutorial
- "Understanding LSTM Networks" http://colah.github.io/posts/2015-08-Understanding-LSTMs/ (**highly recommended!**)
- "LSTM and GRU -- Formula Summary" https://isaacchanghau.github.io/post/lstm-gru-formula/
- "Recurrent neural networks and LSTM tutorial in Python and TensorFlow" http://adventuresinmachinelearning.com/recurrent-neural-networks-lstm-tutorial-tensorflow/
- "How to use an Encoder-Decoder LSTM to Echo Sequences of Random Integers" https://machinelearningmastery.com/how-to-use-an-encoder-decoder-lstm-to-echo-sequences-of-random-integers/
- CS224D Lecture Notes 4 https://cs224d.stanford.edu/lecture_notes/LectureNotes4.pdf